## 4. Register CSV Data With Athena

Code learned and followed from AAI 540 Lab 2

In [2]:
!pip install -q PyAthena

In [3]:
import boto3
import sagemaker
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [19]:
import pandas as pd

# Load the CSV from S3 and parse the 'date' column
df = pd.read_csv(
    "s3://sagemaker-us-east-1-807494057176/root/AAI-540_Predictive-Maintenance-for-Pharmaceutical-Manufacturing-Equipment/predictive_maintenance_dataset.csv",
    parse_dates=['date']
)

# Convert 'date' to datetime if necessary
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Print the result
print(df[['date']].head())


        date
0 2015-01-01
1 2015-01-01
2 2015-01-01
3 2015-01-01
4 2015-01-01


In [20]:
import pandas as pd

# Convert the 'date' column to datetime if it's not already in the correct format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Format 'date' column to include both date and time in human-readable form
df['date'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(x) else None)

# Check the result
print(df[['date']].head())

                  date
0  2015-01-01 00:00:00
1  2015-01-01 00:00:00
2  2015-01-01 00:00:00
3  2015-01-01 00:00:00
4  2015-01-01 00:00:00


In [21]:
# Save the modified CSV back to S3
df.to_csv("s3://{}/root/AAI-540_Predictive-Maintenance-for-Pharmaceutical-Manufacturing-Equipment/predictive_maintenance_with_timestamp.csv".format(bucket), index=False)


In [22]:
df

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9
0,2015-01-01 00:00:00,S1F01085,0,215630672,55,0,52,6,407438,0,0,7
1,2015-01-01 00:00:00,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01 00:00:00,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01 00:00:00,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01 00:00:00,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
124489,2015-11-02 00:00:00,Z1F0MA1S,0,18310224,0,0,0,10,353705,8,8,0
124490,2015-11-02 00:00:00,Z1F0Q8RT,0,172556680,96,107,4,11,332792,0,0,13
124491,2015-11-02 00:00:00,Z1F0QK05,0,19029120,4832,0,0,11,350410,0,0,0
124492,2015-11-02 00:00:00,Z1F0QL3N,0,226953408,0,0,0,12,358980,0,0,0


In [23]:
# Define the database and table names
database_name = "dsoawsfp"  
table_name_csv = "predictive_maintenance_csv"  

In [24]:
# Drop the table if it already exists in Athena
statement_drop = "DROP TABLE IF EXISTS {}.{}".format(database_name, table_name_csv)

# Execute the drop table statement
pd.read_sql(statement_drop, conn)


/tmp/ipykernel_18/1223875043.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_drop, conn)


""


In [25]:
from pyathena import connect

In [26]:
# Define S3 paths
s3_private_path_predictive_maintenance = "s3://{}/root/AAI-540_Predictive-Maintenance-for-Pharmaceutical-Manufacturing-Equipment/".format(bucket)
s3_staging_dir = "s3://{}/athena/staging".format(bucket)

# Create Athena connection
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# Verify connection
print("Athena connection established.")

Athena connection established.


In [27]:
# SQL statement to create the table in Athena
statement_create = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         date timestamp,
         device string,
         failure int,
         metric1 float,
         metric2 float,
         metric3 float,
         metric4 float,
         metric5 float,
         metric6 float,
         metric7 float,
         metric8 float,
         metric9 float
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_predictive_maintenance
)

# Execute table creation
pd.read_sql(statement_create, conn)



/tmp/ipykernel_18/3167562454.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_create, conn)


""


In [28]:
# Query the table with 'timestamp'
statement_query = """SELECT * FROM {}.{} LIMIT 960""".format(database_name, table_name_csv)
df_query = pd.read_sql(statement_query, conn)

# Check the result
print(df_query.head())


/tmp/ipykernel_18/1750700866.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql(statement_query, conn)


        date    device  failure      metric1  metric2  metric3  metric4  \
0 2015-01-01  S1F01085        0  215630672.0     55.0      0.0     52.0   
1 2015-01-01  S1F0166B        0   61370680.0      0.0      3.0      0.0   
2 2015-01-01  S1F01E6Y        0  173295968.0      0.0      0.0      0.0   
3 2015-01-01  S1F01JE0        0   79694024.0      0.0      0.0      0.0   
4 2015-01-01  S1F01R2B        0  135970480.0      0.0      0.0      0.0   

   metric5   metric6  metric7  metric8  metric9  
0      6.0  407438.0      0.0      0.0      7.0  
1      6.0  403174.0      0.0      0.0      0.0  
2     12.0  237394.0      0.0      0.0      0.0  
3      6.0  410186.0      0.0      0.0      0.0  
4     15.0  313173.0      0.0      0.0      3.0  


In [29]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>